In [1]:
from skillcorner.client import SkillcornerClient

import pandas as pd

import numpy as np

import os

secret_password = os.getenv("mdp_skillcorner")
client = SkillcornerClient(username = "Nathan.talbot@etu.uca.fr", password = secret_password)

In [42]:
liste_dico = [{"match_drop" : [1550555, 1546206],
                "ranking" : ["AJ Auxerre", "Angers SCO", "AS Saint-Étienne", "Rodez Aveyron", "Paris FC", "SM Caen", "Stade Lavallois Mayenne FC",
           "Amiens Sporting Club", "En Avant de Guingamp", "Pau FC", "Grenoble Foot 38", "Girondins de Bordeaux", "SC Bastia",
           "FC Annecy", "AC Ajaccio", "Dunkerque", "ES Troyes AC", "US Quevilly-Rouen", "US Concarneau", "Valenciennes FC"],
           "annee" : "2023_2024"},
           {"match_drop" : [],
                "ranking" : ["Le Havre AC", "FC Metz", "Girondins de Bordeaux", "SC Bastia", "SM Caen", "En Avant de Guingamp", "Paris FC",
           "AS Saint-Étienne", "FC Sochaux-Montbéliard", "Grenoble Foot 38", "US Quevilly-Rouen", "Amiens Sporting Club", "Pau FC",
           "Rodez Aveyron", "Stade Lavallois Mayenne FC", "Valenciennes FC", "FC Annecy", "Dijon FCO", "Nîmes Olympique", "Chamois Niortais FC"],
           "annee" : "2022_2023"},
           {"match_drop" : [129364, 128946],
                "ranking" : ["Toulouse FC", "AC Ajaccio", "AJ Auxerre", "Paris FC", "FC Sochaux-Montbéliard", "En Avant de Guingamp",
                             "SM Caen", "Le Havre AC", "Nîmes Olympique", "Pau FC", "Dijon FCO", "SC Bastia", "Chamois Niortais FC", 
                             "Amiens Sporting Club", "Grenoble Foot 38", "Valenciennes FC", "Rodez Aveyron", "US Quevilly-Rouen",
                             "Dunkerque", "AS Nancy-Lorraine"],
           "annee" : "2021_2022"}
           ]
dico = liste_dico[0]

In [43]:
data_import = pd.read_excel(f"../../../../Data_file/Métriques Team sur une Saison Ligue 2 SB + SK/{dico["annee"]}/Skill Corner/data_passes.xlsx", index_col = 0)


In [97]:
data = data_import[~(data_import.match_id.isin(dico["match_drop"]))]
data = data[data.quality_check == True]
data.fillna(0, inplace = True)

nb_matchs = pd.Series(index = data.team_name.unique())
for team in nb_matchs.index :
    nb_matchs[team] = len(data[data.team_name == team].match_id.unique())
nb_matchs = nb_matchs.reindex(dico["ranking"])

drop = ["quality_check", "player_id", "player_name", "short_name", "player_birthdate", "match_name", "match_date", "team_id",
    "competition_id", "competition_name", "season_id", "season_name", "competition_edition_id", "position", "group", "result", "venue",
    "third", "channel", "adjusted_min_tip_per_match"]
data.drop(drop, inplace = True, axis = 1)
sample = data.columns[["sample" in i for i in data.columns]]
data.drop(sample, inplace = True, axis = 1)

# AJOUT NUMÉRO DE JOURNÉE AU DATAFRAME
match_round = pd.read_excel(f"../../../../Data_file/Métriques Team sur une Saison Ligue 2 SB + SK/{dico["annee"]}/Skill Corner/match_round.xlsx",
                            index_col = 0)
match_round.drop(index = dico["match_drop"], inplace = True)
data = data.merge(match_round, right_on = "id", left_on = "match_id").drop("match_id", axis = 1)

# modif métriques ratios
met_ratio = data.columns[["ratio" in i for i in data.columns]]
data = data.groupby(["Journée", "team_name"])
nb_joueur_match = data.apply(len)
data = data.sum()
data[met_ratio] = data[met_ratio].divide(nb_joueur_match, axis = 0)
nb_minute_match = data.pop("minutes_played_per_match")
data = data.multiply(900/nb_minute_match, axis = 0)
moyenne_journée = data.groupby("Journée").mean()
data.to_excel(f"Métriques discriminantes/Tableau métriques/Evolutions métriques/Par journée/{dico["annee"]}/Skill Corner/moyenne_équipe.xlsx")
moyenne_journée.to_excel(f"Métriques discriminantes/Tableau métriques/Evolutions métriques/Par journée/{dico["annee"]}/Skill Corner/moyenne_journée.xlsx")

C:\Users\fabie\AppData\Local\Temp\ipykernel_15980\3230834677.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nb_joueur_match = data.apply(len)
